In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import optuna
from tensorflow.keras.callbacks import EarlyStopping

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
X_train = np.load('/kaggle/input/trainvalidationtest/Train Val Test/X_train.npy', allow_pickle=True)
y_train = np.load('/kaggle/input/trainvalidationtest/Train Val Test/y_train.npy', allow_pickle=True)
X_val = np.load('/kaggle/input/trainvalidationtest/Train Val Test/X_val.npy', allow_pickle=True)
y_val = np.load('/kaggle/input/trainvalidationtest/Train Val Test/y_val.npy', allow_pickle=True)
X_test = np.load('/kaggle/input/trainvalidationtest/Train Val Test/X_test.npy', allow_pickle=True)
y_test = np.load('/kaggle/input/trainvalidationtest/Train Val Test/y_test.npy', allow_pickle=True)

In [3]:
data_info = np.load('/kaggle/input/alllabels/labels.npy', allow_pickle=True)
file_paths = [item[0] for item in data_info]
labels = [item[1] for item in data_info]

In [4]:
label_to_index = {label: index for index, label in enumerate(np.unique(labels))}
y_train_encoded = np.array([label_to_index[label] for label in y_train])
y_val_encoded = np.array([label_to_index[label] for label in y_val])
y_test_encoded = np.array([label_to_index[label] for label in y_test])

num_classes = len(label_to_index)
y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)
y_val_encoded = to_categorical(y_val_encoded, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)

In [5]:
X_train = pad_sequences(X_train, dtype='float32', padding='post', value=0.0)
X_val = pad_sequences(X_val, dtype='float32', padding='post', value=0.0)
X_test = pad_sequences(X_test, dtype='float32', padding='post', value=0.0)

In [6]:
X_train = np.vstack(X_train)
X_val = np.vstack(X_val)
X_test = np.vstack(X_test)

In [7]:
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train_encoded_tensor = tf.convert_to_tensor(y_train_encoded, dtype=tf.float32)
y_val_encoded_tensor = tf.convert_to_tensor(y_val_encoded, dtype=tf.float32)
y_test_encoded_tensor = tf.convert_to_tensor(y_test_encoded, dtype=tf.float32)

In [8]:
signal_length = 2048

In [9]:
X_train_reshaped = X_train.reshape(-1, signal_length, 1)
X_val_reshaped = X_val.reshape(-1, signal_length, 1)
X_test_reshaped = X_test.reshape(-1, signal_length, 1)

In [10]:
mean = np.mean(X_train_reshaped)
std = np.std(X_train_reshaped)

X_train_normalized = (X_train_reshaped - mean) / std
X_val_normalized = (X_val_reshaped - mean) / std
X_test_normalized = (X_test_reshaped - mean) / std

In [ ]:
def objective(trial):
    num_blocks = trial.suggest_int('num_blocks', 3, 5)
    filters = [32, 64, 128, 256] * num_blocks
    kernel_sizes = [3, 5, 8, 16] * num_blocks
    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 2)
    dense_units = [64, 128, 256] * num_dense_layers
    #learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-2)
    num_epochs = trial.suggest_int('num_epochs', 10, 50, step=10)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model = keras.Sequential()
    
    for i in range(num_blocks):
        model.add(layers.Conv1D(filters[i], kernel_sizes[i], activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv1D(filters[i], kernel_sizes[i], activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv1D(filters[i], kernel_sizes[i], activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling1D(pool_size=2))
    
    model.add(layers.Flatten())
    
    for units in dense_units:
        model.add(layers.Dense(units, activation='relu'))
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train_normalized, y_train_encoded,
                        validation_data=(X_val_normalized, y_val_encoded),
                        epochs=num_epochs, batch_size=batch_size,
                        callbacks=[early_stopping], verbose=0)

    return history.history['val_accuracy'][-1]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  

best_params = study.best_params

print("Best Hyperparameters:")
print(best_params)

[I 2023-09-29 00:07:53,041] A new study created in memory with name: no-name-c2150752-5a0b-4447-908f-71b95fec14c1
[I 2023-09-29 00:47:57,209] Trial 0 finished with value: 0.9506000280380249 and parameters: {'num_blocks': 3, 'num_dense_layers': 1, 'num_epochs': 30, 'batch_size': 256}. Best is trial 0 with value: 0.9506000280380249.
[I 2023-09-29 01:37:09,625] Trial 1 finished with value: 0.6907666921615601 and parameters: {'num_blocks': 3, 'num_dense_layers': 2, 'num_epochs': 20, 'batch_size': 128}. Best is trial 0 with value: 0.9506000280380249.
[I 2023-09-29 03:23:54,872] Trial 2 finished with value: 0.9987333416938782 and parameters: {'num_blocks': 5, 'num_dense_layers': 2, 'num_epochs': 30, 'batch_size': 64}. Best is trial 2 with value: 0.9987333416938782.
[I 2023-09-29 04:58:09,439] Trial 3 finished with value: 0.7482666373252869 and parameters: {'num_blocks': 4, 'num_dense_layers': 1, 'num_epochs': 20, 'batch_size': 64}. Best is trial 2 with value: 0.9987333416938782.
[I 2023-09-2